In [2]:
import pandas as pd
import numpy as np
import requests
import requests_cache
from requests_oauthlib import OAuth1Session
import json

In [3]:
# Requests Cache is a great way to save on API calls
# http://requests-cache.readthedocs.io/
requests_cache.install_cache()

In [4]:
# Link to your own JSON credentials file
credentials = json.loads(open('oauth.txt').read())

twitter = OAuth1Session(client_key=credentials["client_key"],
                    client_secret=credentials["client_secret"],
                    resource_owner_key=credentials["resource_owner_key"],
                    resource_owner_secret=credentials["resource_owner_secret"])

In [5]:
candidates = ["realdonaldtrump", "hillaryclinton"]

for candidate in candidates:

    base_url = "https://api.twitter.com/1.1/statuses/user_timeline.json?screen_name=%s&count=100" % candidate
    tweets = pd.DataFrame(columns=["id", "text", "created_at", "favorite_count", 'retweet_count']) # We add to this DF
    maxid = str(twitter.get(base_url).json()[0]['id']) # Start with the most recent Tweet id

    for j in range(0, 35): # 35, because 35 * 100 Tweets per call is just more than the 3000 we're able to access
        r = twitter.get(base_url+"&max_id="+maxid)
        for i in r.json():
            # Add to our Tweets DataFrame
            tweets.loc[tweets.shape[0]] = [str(i["id"]), i["text"], i["created_at"], i["favorite_count"], i['retweet_count']]
            maxid = str(i["id"]) # Take the oldest Tweet id for use in getting the next 100 Tweets
    
    tweets = tweets.drop_duplicates(subset=["id"])
    tweets.to_csv("%s_tweets.csv" % candidate, encoding='utf-8') 